In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
tables = pd.read_html("https://www.movingsolutions.in/blog/2020/02/05/pin-codes-of-delhi-locations/",header=0,index_col=0)
Borough = ['East Delhi', 'West delhi', 'South Delhi', 'North Delhi', 'New Delhi', 'Central Delhi', 'West Delhi', 'South West Delhi']

In [3]:
for i in range(len(Borough)):
    tables[i]['Borough']=Borough[i]  
DF = tables[0]    
for j in range(len(Borough)-1):
    DF = pd.concat([DF, tables[j+1]])

DF = DF.reset_index()
DF.rename(columns = {'Location':'Neighborhood','Pin code':'Postalcode'}, inplace = True)    
DF.head()

,Neighborhood,Postalcode,Borough
0,Anand Vihar,110092,East Delhi
1,Azad Nagar,110051,East Delhi
2,Babarpur,110032,East Delhi
3,Balbir Nagar,110032,East Delhi
4,Bhajan Pura,110053,East Delhi


In [4]:
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('adarsh nagar,delhi, india')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))


Latitude = 28.7144008, Longitude = 77.1672884


In [5]:
DF.dtypes

Neighborhood    object
Postalcode       int64
Borough         object
dtype: object

In [6]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    #while(lat_lng_coords is None):
    lat_lng_coords = locator.geocode('{}, Delhi, india'.format(neighborhood))
    if(lat_lng_coords is None):
        lat_lng_coords = 'Nan'
    return lat_lng_coords

In [7]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(Neighborhood) for Neighborhood in DF["Neighborhood"].tolist() ]

In [8]:
latitude = []
longitude = []
for data in coords:
    if data != 'Nan':
        latitude.append(data.latitude)
        longitude.append(data.longitude)
    else :
        latitude.append(0)
        longitude.append(0)

In [9]:
DF['Latitude']=latitude
DF['Longitude']=longitude
DF.shape

(493, 5)

In [10]:
DF = DF[DF['Latitude'] != 0]

In [11]:
DF.head()

,Neighborhood,Postalcode,Borough,Latitude,Longitude
0,Anand Vihar,110092,East Delhi,28.641115,77.312502
1,Azad Nagar,110051,East Delhi,28.662682,77.279515
2,Babarpur,110032,East Delhi,28.687431,77.279755
3,Balbir Nagar,110032,East Delhi,28.683790,77.290754
5,Bhola Nath nagar,110032,East Delhi,28.669127,77.285241


In [12]:
address = 'Delhi, IN'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 28.6517178, 77.2219388.


In [13]:
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for Latitude, Longitude, Borough, Neighborhood in zip(DF['Latitude'], DF['Longitude'], DF['Borough'], DF['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
    
map

In [14]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: WDYUDPBSNVHNSR3NNV54QXP4GSSMGFF0Z5XMRWFPEW2UFL3T
CLIENT_SECRET:YH0RSL4NX0PRPVCJPTCSHPNH3QCOBLPRZHGXKZ0YKPHAZQFS


In [15]:
#Let's get the geographical coordinates of Anand Vihar
neighborhood_latitude = DF.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DF.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = DF.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Anand Vihar are 28.641115, 77.3125024.


In [16]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edfddf65f782d7169d5a26f'},
 'response': {'headerLocation': 'East Delhi',
  'headerFullLocation': 'East Delhi, Delhi',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 28.645615004500005,
    'lng': 77.31762022319207},
   'sw': {'lat': 28.636614995499993, 'lng': 77.30738457680793}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d4d7a799ee137040d3d848b',
       'name': 'PVR EDM',
       'location': {'lat': 28.641322745064787,
        'lng': 77.31702625175842,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.641322745064787,
          'lng': 77.31702625175842}],
        'distance': 442,
        'cc': 'IN',
        'city': 'Ghāziābād',
        'state': 'Uttar Pr

In [18]:
#let's borrow the get_category_type function from the Foursquare lab. function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,PVR EDM,Movie Theater,28.641323,77.317026
1,Pizza Hut,Pizza Place,28.641347,77.317054
2,Lemon Tree Hotel,Hotel,28.641373,77.316549
3,Big Cinema Lounge Ebony,Multiplex,28.641398,77.317402


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
Delhi_venues = getNearbyVenues(names=DF['Neighborhood'],
                   latitudes=DF['Latitude'],
                   longitudes=DF['Longitude']
                 )

Anand Vihar
Azad Nagar
Babarpur
Balbir Nagar
Bhola Nath nagar
Chilla
Dilshad Garden
G.T.B. Hospital
Gandhi Nagar
Gandhi Nagar bazar
Geeta Colony
Ghazipur
Himmatpuri
Jafrabad
Jhilmil
Jhilmil Tahirpur
Johripur
Kailash Nagar
Kalyanpuri
Kalyanvas
Karawal Nagar
Krishna Nagar
Laxmi Nagar
Loni Road
Mandawali Fazalpur
Maujpur
Mayur Vihar Ph 3
Nand Nagri
New Seemapuri
New Usmanpur
Nirman Vihar
Old Seemapuri
Patparganj
Ram Nagar
Seelampur
Shahdara
Shakarpur
Shastri Nagar
Shivaji Park
Sonia Vihar
Surajmal Vihar
Trilokpuri
V.K. Nagar
Vishwas Nagar
Vivek Vihar
Yamuna Vihar
Ashok Nagar
Bakkarwala
Chand Nagar
Chaukhandi
Delhi Industrial Area
Fateh Nagar
Hari Nagar
Rajouri Garden
Jail Road
Janakpuri
Janta Market
Jeevan Park
Jwalapuri
Jwalaheri
Madipur Village
Mahabir Nagar
Matiala
Mundka
Nangloi
Nangloi – II
Nangloi – III
Nilothi
Paschim Vihar
Paschim Vihar b block
Peeragarhi
Punjabi Bagh
Punjabi Bagh Sect 3
Rajouri Market
Ramesh Nagar
Ranhola
Shivaji Park
Subhash Nagar
Subhash Nagar west
Sunder Vihar

In [24]:
print(Delhi_venues.shape)
Delhi_venues.head()

(1556, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Anand Vihar,28.641115,77.312502,PVR EDM,28.641323,77.317026,Movie Theater
1,Anand Vihar,28.641115,77.312502,Pizza Hut,28.641347,77.317054,Pizza Place
2,Anand Vihar,28.641115,77.312502,Lemon Tree Hotel,28.641373,77.316549,Hotel
3,Anand Vihar,28.641115,77.312502,Big Cinema Lounge Ebony,28.641398,77.317402,Multiplex
4,Azad Nagar,28.662682,77.279515,Domino's Pizza,28.660000,77.277000,Pizza Place


In [25]:
Delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A F Palam,1,1,1,1,1,1
Alaknanda,12,12,12,12,12,12
Ali,7,7,7,7,7,7
Aliganj,7,7,7,7,7,7
Alipur,1,1,1,1,1,1
Amar Colony,9,9,9,9,9,9
Amberhai,1,1,1,1,1,1
Anand Niketan,3,3,3,3,3,3
Anand Parbat,2,2,2,2,2,2


In [26]:
print('There are {} uniques categories.'.format(len(Delhi_venues['Venue Category'].unique())))

There are 192 uniques categories.


In [27]:
# one hot encoding
Delhi_onehot = pd.get_dummies(Delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Delhi_onehot['Neighborhood'] = Delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Delhi_onehot.columns[-1]] + list(Delhi_onehot.columns[:-1])
Delhi_onehot = Delhi_onehot[fixed_columns]

Delhi_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Kids Store,Korean Restaurant,Lake,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Portuguese Restaurant,Pub,Punjabi Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tour Provider,Toy / Game Store,Trail,Train Station,Travel Lounge,University,Vegetarian / Vegan Restaurant,Water Park,Women's Store,Yoga Studio
0,Anand Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Anand Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Anand Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Anand Vihar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
Delhi_onehot.shape

(1556, 193)

In [29]:
Delhi_grouped = Delhi_onehot.groupby('Neighborhood').mean().reset_index()
Delhi_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,General Travel,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Kids Store,Korean Restaurant,Lake,Leather Goods Store,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Portuguese Restaurant,Pub,Punjabi Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Sake Bar,Salon / Barbershop,Sandwich Place,Sculpture Garden,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Spiritual Center,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tour Provider,Toy / Game Store,Trail,Train Station,Travel Lounge,University,Vegetarian / Vegan Restaurant,Water Park,Women's Store,Yoga Studio
0,A F Palam,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.0,0.00000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00

In [30]:
Delhi_grouped.shape

(266, 193)

In [31]:
num_top_venues = 5

for hood in Delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Delhi_grouped[Delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A F Palam----
                             venue  freq
0                              Bar   1.0
1                              ATM   0.0
2  Molecular Gastronomy Restaurant   0.0
3              Moroccan Restaurant   0.0
4                           Mosque   0.0


----Alaknanda----
                       venue  freq
0          Indian Restaurant  0.17
1                  BBQ Joint  0.17
2                Pizza Place  0.08
3                 Restaurant  0.08
4  Middle Eastern Restaurant  0.08


----Ali----
               venue  freq
0  Indian Restaurant  0.29
1              Hotel  0.14
2             Hostel  0.14
3      Movie Theater  0.14
4               Road  0.14


----Aliganj----
                        venue  freq
0           Indian Restaurant  0.29
1         Japanese Restaurant  0.14
2               Deli / Bodega  0.14
3               Shopping Mall  0.14
4  Modern European Restaurant  0.14


----Alipur----
                 venue  freq
0                  ATM   1.0
1    Accessories Stor

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Delhi_grouped['Neighborhood']

for ind in np.arange(Delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A F Palam,Bar,Yoga Studio,Golf Course,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
1,Alaknanda,Indian Restaurant,BBQ Joint,Steakhouse,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,New American Restaurant,Coffee Shop,Restaurant,Food & Drink Shop
2,Ali,Indian Restaurant,Road,Hotel,Hostel,Movie Theater,Asian Restaurant,Yoga Studio,Food,Garden,Furniture / Home Store
3,Aliganj,Indian Restaurant,Italian Restaurant,Deli / Bodega,Modern European Restaurant,Shopping Mall,Japanese Restaurant,Duty-free Shop,Food Court,Gastropub,Garden Center
4,Alipur,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck


In [34]:
# set number of clusters
kclusters = 5

Delhi_grouped_clustering = Delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 1, 2, 0, 0, 4, 3, 4], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [48]:
Delhi_merged = DF
Delhi_merged['Postalcode']=DF['Neighborhood']
DF.head()

,Neighborhood,Postalcode,Borough,Latitude,Longitude
0,Anand Vihar,Anand Vihar,East Delhi,28.641115,77.312502
1,Azad Nagar,Azad Nagar,East Delhi,28.662682,77.279515
2,Babarpur,Babarpur,East Delhi,28.687431,77.279755
3,Balbir Nagar,Balbir Nagar,East Delhi,28.683790,77.290754
5,Bhola Nath nagar,Bhola Nath nagar,East Delhi,28.669127,77.285241


In [49]:
Delhi_merged

,Neighborhood,Postalcode,Borough,Latitude,Longitude
0,Anand Vihar,Anand Vihar,East Delhi,28.641115,77.312502
1,Azad Nagar,Azad Nagar,East Delhi,28.662682,77.279515
2,Babarpur,Babarpur,East Delhi,28.687431,77.279755
3,Balbir Nagar,Balbir Nagar,East Delhi,28.683790,77.290754
5,Bhola Nath nagar,Bhola Nath nagar,East Delhi,28.669127,77.285241
7,Chilla,Chilla,East Delhi,28.595984,77.300936
9,Dilshad Garden,Dilshad Garden,East Delhi,28.675826,77.321516
11,G.T.B. Hospital,G.T.B. Hospital,East Delhi,28.684560,77.311077
12,Gandhi Nagar,Gandhi Nagar,East Delhi,28.453175,77.015329
13,Gandhi Nagar bazar,Gandhi Nagar bazar,East Delhi,28.656097,77.235436


In [50]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Delhi_merged = Delhi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Delhi_merged.head() # check the last columns!

,Neighborhood,Postalcode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anand Vihar,Anand Vihar,East Delhi,28.641115,77.312502,4.0,Multiplex,Pizza Place,Hotel,Movie Theater,Yoga Studio,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
1,Azad Nagar,Azad Nagar,East Delhi,28.662682,77.279515,0.0,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Park,Donut Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
2,Babarpur,Babarpur,East Delhi,28.687431,77.279755,0.0,Electronics Store,Motorcycle Shop,Park,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
3,Balbir Nagar,Balbir Nagar,East Delhi,28.683790,77.290754,1.0,Indian Restaurant,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
5,Bhola Nath nagar,Bhola Nath nagar,East Delhi,28.669127,77.285241,0.0,Pharmacy,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck


In [51]:
Delhi_merged = Delhi_merged.dropna()
Delhi_merged

,Neighborhood,Postalcode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anand Vihar,Anand Vihar,East Delhi,28.641115,77.312502,4.0,Multiplex,Pizza Place,Hotel,Movie Theater,Yoga Studio,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
1,Azad Nagar,Azad Nagar,East Delhi,28.662682,77.279515,0.0,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Park,Donut Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
2,Babarpur,Babarpur,East Delhi,28.687431,77.279755,0.0,Electronics Store,Motorcycle Shop,Park,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
3,Balbir Nagar,Balbir Nagar,East Delhi,28.683790,77.290754,1.0,Indian Restaurant,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
5,Bhola Nath nagar,Bhola Nath nagar,East Delhi,28.669127,77.285241,0.0,Pharmacy,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
7,Chilla,Chilla,East Delhi,28.595984,77.300936,4.0,Hotel,Restaurant,Arcade,Bakery,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store
9,Dilshad Garden,Dilshad Garden,East Delhi,28.675826,77.321516,1.0,Diner,Indian Restaurant,Pizza Place,Light Rail Station,Duty-free Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
11,G.T.B. Hospital,G.T.B. Hospital,East Delhi,28.684560,77.311077,0.0,Gym,Market,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
12,Gandhi Nagar,Gandhi Nagar,East Delhi,28.453175,77.015329,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
13,Gandhi Nagar bazar,Gandhi Nagar bazar,East Delhi,28.656097,77.235436,1.0,Indian Restaurant,Snack Place,Market,Hotel,Flea Market,Fast Food Restaurant,Dessert Shop,Museum,Food & Drink Shop,Garden Center


In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Delhi_merged['Latitude'], Delhi_merged['Longitude'], Delhi_merged['Neighborhood'], Delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 0, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Azad Nagar,0.0,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Park,Donut Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
2,Babarpur,0.0,Electronics Store,Motorcycle Shop,Park,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
5,Bhola Nath nagar,0.0,Pharmacy,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
11,G.T.B. Hospital,0.0,Gym,Market,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
14,Geeta Colony,0.0,ATM,Pizza Place,Gym,Dessert Shop,Airport,Food & Drink Shop,General Travel,Gastropub,Garden Center,Garden
15,Ghazipur,0.0,Antique Shop,Flower Shop,Yoga Studio,Food & Drink Shop,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
21,Jafrabad,0.0,IT Services,Motorcycle Shop,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
23,Jhilmil,0.0,Metro Station,Yoga Studio,Gift Shop,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
26,Kailash Nagar,0.0,Clothing Store,Yoga Studio,Food & Drink Shop,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
28,Kalyanvas,0.0,Sporting Goods Shop,Gym,Moving Target,Yoga Studio,Food & Drink Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


In [54]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 1, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Balbir Nagar,1.0,Indian Restaurant,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
9,Dilshad Garden,1.0,Diner,Indian Restaurant,Pizza Place,Light Rail Station,Duty-free Shop,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
13,Gandhi Nagar bazar,1.0,Indian Restaurant,Snack Place,Market,Hotel,Flea Market,Fast Food Restaurant,Dessert Shop,Museum,Food & Drink Shop,Garden Center
20,Himmatpuri,1.0,ATM,Indian Restaurant,Electronics Store,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
46,Patparganj,1.0,Indian Restaurant,Department Store,Coffee Shop,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
76,Fateh Nagar,1.0,Indian Restaurant,Indian Sweet Shop,Ice Cream Shop,Pizza Place,Fried Chicken Joint,Food Court,Bakery,Punjabi Restaurant,Yoga Studio,Flower Shop
83,Janta Market,1.0,Indian Restaurant,Juice Bar,Electronics Store,Food Truck,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store
104,Peeragarhi,1.0,Indian Restaurant,Convenience Store,Hotel Bar,Yoga Studio,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
117,Uttam Nagar,1.0,Indian Restaurant,Gym,Hotel,Light Rail Station,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
118,Vikas Puri,1.0,Indian Restaurant,Sandwich Place,Playground,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


In [55]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 2, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Gandhi Nagar,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
27,Kalyanpuri,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
58,Sonia Vihar,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
70,Bakkarwala,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
101,Nilothi,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
157,Lal Kuan,2.0,ATM,Food & Drink Shop,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
172,Pragati Vihar,2.0,ATM,Sandwich Place,Dim Sum Restaurant,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
273,Alipur,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
278,Bakhtawar Pur,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
281,Barwala,2.0,ATM,Women's Store,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck


In [56]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 3, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Old Seemapuri,3.0,Business Service,Fast Food Restaurant,Yoga Studio,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
68,Ashok Nagar,3.0,Fast Food Restaurant,Café,North Indian Restaurant,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
105,Punjabi Bagh,3.0,Fast Food Restaurant,Garden Center,Sandwich Place,Pizza Place,Bar,Gym / Fitness Center,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store
106,Punjabi Bagh Sect 3,3.0,Fast Food Restaurant,Garden Center,Sandwich Place,Pizza Place,Bar,Gym / Fitness Center,Yoga Studio,Food & Drink Shop,Garden,Furniture / Home Store
113,Sunder Vihar,3.0,ATM,Café,Fast Food Restaurant,Clothing Store,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
115,Tilak Nagar,3.0,Fast Food Restaurant,Bookstore,Indian Restaurant,Clothing Store,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store
116,Tilak Nagar east,3.0,Fast Food Restaurant,Bookstore,Indian Restaurant,Clothing Store,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store
163,Masjid Moth,3.0,Pizza Place,Fast Food Restaurant,Sporting Goods Shop,Café,Yoga Studio,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
177,Sadiq Nagar,3.0,Café,Fast Food Restaurant,Park,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
200,Delhi Sadar Bazar,3.0,Fast Food Restaurant,Yoga Studio,Food & Drink Shop,General Travel,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


In [57]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 4, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,Postalcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Anand Vihar,4.0,Multiplex,Pizza Place,Hotel,Movie Theater,Yoga Studio,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food Truck
7,Chilla,4.0,Hotel,Restaurant,Arcade,Bakery,Yoga Studio,Food & Drink Shop,Gastropub,Garden Center,Garden,Furniture / Home Store
49,Ram Nagar,4.0,Hotel,Pizza Place,Restaurant,Indian Restaurant,Motel,Hostel,Coffee Shop,Fast Food Restaurant,Yoga Studio,Flower Shop
77,Hari Nagar,4.0,Other Great Outdoors,Hotel,Restaurant,Arcade,Stadium,Yoga Studio,Food,Garden,Furniture / Home Store,Frozen Yogurt Shop
129,Badarpur,4.0,Train Station,IT Services,Hotel,Art Gallery,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
131,Chirag Delhi,4.0,Yoga Studio,Motel,Trail,Hotel,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
152,Kalkaji,4.0,Dessert Shop,Sandwich Place,Pizza Place,Hotel,Farmers Market,Flower Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
155,Krishna Market,4.0,Indian Restaurant,Pharmacy,Hotel,Lake,Yoga Studio,Flower Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
171,Panchsheel Enclave,4.0,Lounge,Indian Restaurant,Event Space,Hotel,Gym,Food,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop
223,Bank Street,4.0,Snack Place,Dessert Shop,Hotel,Food,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
